## Due to the 100 MB size limit on GitHub, I separated the data cleaning processes into three notebooks:

* sf_crime_prediction.ipynb (main notebook)
* working_on_weather_data.ipynb
* working_on_flight_data.ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import requests
import os
from io import StringIO
import json

import missingno as msno
import folium


from openai import OpenAI
from dotenv import load_dotenv
import re

from folium.plugins import MarkerCluster

pd.set_option('display.max_columns', None)

In [ ]:
path = os.getcwd()
raw_data_path = path + "/../data/raw/"


In [ ]:
# download our incident data and store in the raw data directory

base_url = "https://data.sfgov.org/resource/wg3w-h783.csv"
limit  = 1000
offset = 0
data  = []

df_incident = pd.DataFrame()

data_path = raw_data_path +'incident_raw.csv'

if not os.path.exists(data_path):
  while True:
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    response = requests.get(url)
    if response.status_code == 200:
      if not offset%100000:
        print(f"Fetching data from {url}")
      batch_df = pd.read_csv(StringIO(response.text))
      if batch_df.empty:
        break
      df_incident = pd.concat([df_incident, batch_df], ignore_index=True)
      offset += limit
    else:
      print('Done!')
      break
  df_incident.to_csv(data_path)
  

In [ ]:
# create dataframe
df_incident = pd.read_csv(data_path)

print(df_incident.info())
print("")
df_incident.head()

# Metadata

<div class="schema-column-preview-table">
  <table class="forge-table">
    <thead>
      <tr>
        <th>Column Name</th>
        <th>Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Incident Datetime</td>
        <td>The date and time when the incident occurred</td>
      </tr>
      <tr>
        <td>Incident Date</td>
        <td>The date the incident occurred</td>
      </tr>
      <tr>
        <td>Incident Time</td>
        <td>The time the incident occurred</td>
      </tr>
      <tr>
        <td>Incident Year</td>
        <td>The year the incident occurred, provided as a convenience for filtering</td>
      </tr>
      <tr>
        <td>Incident Day of Week</td>
        <td>The day of the week the incident occurred</td>
      </tr>
      <tr>
        <td>Report Datetime</td>
        <td>Distinct from Incident Datetime, Report Datetime is when the report was filed</td>
      </tr>
      <tr>
        <td>Row ID</td>
        <td>A unique identifier for each row of data in the dataset</td>
      </tr>
      <tr>
        <td>Incident ID</td>
        <td>This is the system generated identifier for incident reports</td>
      </tr>
      <tr>
        <td>Incident Number</td>
        <td>The number issued on the report, used to reference cases and report documents</td>
      </tr>
      <tr>
        <td>CAD Number</td>
        <td>The Computer Aided Dispatch (CAD) is the system used by the Department of Emergency Management to dispatch officers and other public safety personnel</td>
      </tr>
      <tr>
        <td>Report Type Code</td>
        <td>A system code for report types</td>
      </tr>
      <tr>
        <td>Report Type Description</td>
        <td>The description of the report type, such as Initial, Initial Supplement, Vehicle Initial, etc.</td>
      </tr>
      <tr>
        <td>Filed Online</td>
        <td>Indicates if the report was filed online by the public using SFPD’s self-service reporting system</td>
      </tr>
      <tr>
        <td>Incident Code</td>
        <td>System codes to describe a type of incident</td>
      </tr>
      <tr>
        <td>Incident Category</td>
        <td>A category mapped onto the Incident Code used in statistics and reporting</td>
      </tr>
      <tr>
        <td>Incident Subcategory</td>
        <td>A subcategory mapped to the Incident Code used for statistics and reporting</td>
      </tr>
      <tr>
        <td>Incident Description</td>
        <td>Description of the incident that corresponds with the Incident Code</td>
      </tr>
      <tr>
        <td>Resolution</td>
        <td>The resolution of the incident at the time of the report</td>
      </tr>
      <tr>
        <td>Intersection</td>
        <td>The 2 or more street names that intersect closest to the original incident</td>
      </tr>
      <tr>
        <td>CNN</td>
        <td>The unique identifier of the intersection for reference back to other related basemap datasets</td>
      </tr>
      <tr>
        <td>Police District</td>
        <td>The Police District where the incident occurred</td>
      </tr>
      <tr>
        <td>Analysis Neighborhood</td>
        <td>The neighborhood where each incident occurs</td>
      </tr>
      <tr>
        <td>Supervisor District</td>
        <td>Current Supervisor District</td>
      </tr>
      <tr>
        <td>Supervisor District 2012</td>
        <td>Previous 2012-2022 Supervisor District</td>
      </tr>
      <tr>
        <td>Latitude</td>
        <td>The latitude coordinate in WGS84</td>
      </tr>
      <tr>
        <td>Longitude</td>
        <td>The longitude coordinate in WGS84</td>
      </tr>
      <tr>
        <td>Point</td>
        <td>Geolocation in OGC WKT format</td>
      </tr>
    </tbody>
  </table>
</div>


In [ ]:
# print the number of missing data for each column
df_isna = pd.DataFrame(df_incident.isna().sum(), columns=['missing_data'])
df_isna[df_isna['missing_data']!=0]

In [ ]:
del df_isna # no need to store extra data in the memory

In [ ]:
msno.matrix(df_incident.sort_values(by='Incident Datetime'), figsize=(20, 10))
plt.show()

### columns: 
* `ESNCAG - Boundary File` 
* `Central Market/Tenderloin Boundary Polygon - Updated` 
* `Civic Center Harm Reduction Project Boundary` 
* `HSOC Zones as of 2018-06-05`
* `Invest In Neighborhoods (IIN) Areas` 
### are pretty much empty columns, those columns will not help with any prediction.

## We can start cleaning our data with dropping those columns 


In [ ]:
df_incident.drop(columns=['ESNCAG - Boundary File', 'Central Market/Tenderloin Boundary Polygon - Updated', 'Civic Center Harm Reduction Project Boundary', 'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas'], inplace=True)

In [ ]:
msno.matrix(df_incident.sort_values(by='Incident Datetime'), figsize=(20, 10))
plt.show()

# Check for duplicates

In [ ]:
# a basic check for duplicates with checking Date, id and incident code 
# Later we will see there are duplicates for incident ID, and we will figure out that mistery
duplicates = df_incident[['Incident Date', 'Incident ID',  'Incident Code']].duplicated(keep=False).sum()
print(f"There are {duplicates} duplicates in the dataframe.")

# Which columns we will drop?
### Lets focus on the features we need for now!

### keep:
* Incident Date
* Incident Time
* Incident Day of Week
* Incident Category 
* Incident Subcategory 
* Incident Description
* Incident Code
* Incident ID
* Intersection 
* Latitude
* Longtitude
* Analysis Neighborhood
* Report type code
* report type description




In [ ]:
df_incident  = df_incident.drop(columns=['Incident Datetime',  'Report Datetime', 'Row ID', 'Resolution', 'Report Type Code',	'Report Type Description','CNN', 'Police District', 'Supervisor District', 'Supervisor District 2012', 'Neighborhoods','Current Supervisor Districts', 'Current Police Districts', 'Point', 'Filed Online']).sort_values(by='Incident Date')

In [ ]:
# `CAD Number` will not be helpful to our model and we can drop that column.
# Same for the 'Intersection'  we will continue with 'Analysis Neighborhood' column for location 

df_incident.drop(columns=['CAD Number', 'Intersection'], inplace=True)
df_incident.isna().sum()

In [ ]:
# Check for any incident codes that are present in both categories: 
# 1. Incident codes with missing 'Incident Category'
# 2. Incident codes with non-missing 'Incident Category'
df_incident[df_incident['Incident Code'].isin(set(df_incident[df_incident['Incident Category'].isna()]['Incident Code']))]['Incident Category'].notna().sum()

## There are some Incident categories is NaN and I wanted to check if corresponding Incident ID has any corresponding Incident Category not null. 
## But looks like there is none.  Look like we can fill these missing categories according to Incident  Description.

In [ ]:
df_incident[df_incident['Incident Code'].isin(set(df_incident[df_incident['Incident Category'].isna()]['Incident Code']))]['Incident Code'].value_counts()

In [ ]:
df_incident[df_incident['Incident Code'].isin(set(df_incident[df_incident['Incident Category'].isna()]['Incident Code']))]['Incident Code'].value_counts().sum()

In [ ]:
df_incident[df_incident['Incident Code'] == 65021]['Incident Category'].sum()

## After some exploaration I found out the incident codes above has no corresponding Incident category, and lets see what are the corresponding Incidinet Description 


In [ ]:
descriptions_for_missing_categories = df_incident[df_incident['Incident Code'].isin(set(df_incident[df_incident['Incident Category'].isna()]['Incident Code']))]['Incident Description'].unique()
descriptions_for_missing_categories

In [ ]:
years_for_missing_categories = df_incident[df_incident['Incident Code'].isin(set(df_incident[df_incident['Incident Category'].isna()]['Incident Code']))]['Incident Date']
pd.to_datetime(years_for_missing_categories).dt.year.value_counts()
# I could not find any correlations between year and missing data at category

In [ ]:
for missing_description in descriptions_for_missing_categories:
    print(f"{missing_description}: ", end=" ")
    print(df_incident[(df_incident['Incident Description'] == missing_description) & (df_incident['Incident Category'].notna())].value_counts().sum())

### `Theft, Boat` from description is the only one has been in a category before.
### Lets see what is that category:

In [ ]:
df_incident[df_incident['Incident Description']=='Theft, Boat']

### We can change Incident code to 'Larceny Theft' for corresponding to  where 'Incident Description" is 'Theft, Boat. 

In [ ]:
df_incident.loc[df_incident['Incident Description']=='Theft, Boat',['Incident Category', 'Incident Subcategory']] = 'Larceny Theft'


## For the remaining missing data, I will use AI (OpenAI) to classify the corresponding Incident Category.

In [ ]:
load_dotenv()

client = OpenAI(api_key=str(os.getenv('OPENAI_API_KEY')))


model = 'gpt-4o'

instructions = f"""You will receive a list of crime descriptions, and you will find the best possible corresponding categories from 
{set(df_incident[df_incident['Incident Category'].notnull()]['Incident Category'])}.
Your response should be in the format of a Python dictionary with description:category pairs. with not using new line, just one line"""

def get_comtpetion(prompt, model=model, instruction=instructions):
    message=[{'role':'system', 'content':instruction}, 
             {'role':'user', 'content':prompt}]
    response=client.chat.completions.create(
        model=model,
        messages=message,
        temperature = 1,
        
    )
    return response.choices[0].message.content

missing_descriptions = set(df_incident[df_incident['Incident Code'].isin(set(df_incident[df_incident['Incident Category'].isna()]['Incident Code']))]['Incident Description'])



text_dict = get_comtpetion(f"Here are the descriptions for you: {list(missing_descriptions)}")

match = re.search(r"\{(.*?)\}", text_dict.strip(), re.DOTALL) 
if match:
    dict_text = '{'+match.group(1)+'}'
else:
    print('No match')


In [ ]:
description_to_category =eval(dict_text)
for key, value in description_to_category.items():
    df_incident.loc[df_incident['Incident Description'] == key , ['Incident Category', 'Incident Subcategory']] = value

df_incident.isna().sum()    

## Let's Fill Empty Cells in Latitude, Longitude, and 'Analysis Neighborhood' Columns

### My Plan:

1. **Filling 'Analysis Neighborhood':**
   - Start by filling the 'Analysis Neighborhood' column with the mode (most frequent) neighborhood for the corresponding incident category.

2. **Filling Latitude and Longitude:**
   - Fill the Latitude and Longitude columns with the mean values of their respective groups, grouped by the first neighborhood and incident category.


In [ ]:

mode_neighborhoods = df_incident.groupby('Incident Category')['Analysis Neighborhood'].agg(lambda x: x.mode()[0])

cat_neig = mode_neighborhoods.to_dict()


for index, row in df_incident.iterrows():
    if pd.isna(row['Analysis Neighborhood']):
        df_incident.loc[index, 'Analysis Neighborhood'] = cat_neig[row['Incident Category']]
        
   

In [ ]:
df_incident['Latitude'] = df_incident.groupby('Analysis Neighborhood')['Latitude'].transform(lambda x: x.fillna(x.mean()))
df_incident['Longitude'] = df_incident.groupby('Analysis Neighborhood')['Longitude'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_incident.isna().sum()

In [ ]:
# finally fix the data type for 'Incident Date' 
df_incident['Incident Date'] = pd.to_datetime(df_incident['Incident Date'])

In [ ]:
plt.figure(figsize=(8,6))
category_counts = df_incident['Incident Category'].value_counts()
sorted_categories = category_counts.index.to_list()
sns.countplot(data=df_incident,
             x='Incident Category', order=sorted_categories)
plt.title("Incident Categories Density in Order from Highest to Lowest")
plt.xticks(rotation=90)
plt.show();


## There are too many different Incident categories which are not being so helpful. I will focus on that later

In [ ]:
# lets check consistincy
df_incident[df_incident['Incident Description'] == 'Battery']['Incident Category'].value_counts()

## Lets merge our dataframes (df_incident & df_weather)

In [ ]:
df_weather = pd.read_csv(path + "/../data/processed/sf_weather_processed.csv")
df_weather['DATE']= pd.to_datetime(df_weather['DATE'], format="%Y-%m-%d")



In [ ]:
# we will merge our data on date coumns

df_incident = pd.merge(df_incident, df_weather, left_on='Incident Date', right_on='DATE', how='left')

In [ ]:
del df_weather
df_incident = df_incident.drop(columns=['DATE'])
df_incident.head()

## Lets rename our columns for consistency.

In [ ]:
df_incident.rename(columns={
    'Incident Date': 'incident_date',
    'Incident Time': 'incident_time',
    'Incident Year': 'incident_year',
    'Incident Day of Week': 'incident_day',
    'Incident ID': 'incident_id',
    'Incident Number': 'incident_no',
    'Incident Code': 'incident_code',
    'Incident Category': 'category',
    'Incident Subcategory': 'subcategory',
    'Incident Description': 'description',
    'Analysis Neighborhood': 'neighborhood',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'PRCP': 'precipitation',
    'TMIN': 'min_temperature',
    'TMAX': 'max_temperature'
}, inplace=True)

df_incident.head(1)

## Looks good! Now time to get our flight data!

In [ ]:
df_flight = pd.read_csv(path+"/../data/processed/flight_processed.csv")
df_flight['activity_period'] = pd.to_datetime(df_flight['activity_period'], format="%Y-%m-%d")

# Merge flight data

In [ ]:
df_incident = df_incident.merge(df_flight, left_on='incident_date', right_on='activity_period', how='left')
del df_flight

In [ ]:
df_incident[['activity_period','deplaned_passenger','enplaned_passenger']] = df_incident[['activity_period','deplaned_passenger','enplaned_passenger']].ffill()

In [ ]:
df_incident.isna().sum()

In [ ]:
# we have missing data on weather related columns Lets see what are the corresponding dates to those

df_incident[df_incident['precipitation'].isna()]['incident_date'].value_counts()

### We only have data missing for 4 days related to weather. It will be acceptable to fill these cells with data from earlier days, as the weather conditions are likely to be quite similar.

In [ ]:
df_incident.ffill(inplace=True)

In [ ]:
df_incident.drop(columns='activity_period', inplace=True)
df_incident.head()

In [ ]:
df_incident.isna().sum()

## Perfect! We merged all there tables and we handled all the missing datas.

## Now we need to work on incidents. Many incidents consist of multiple sub-incidents. We need to filter these incidents into one main incident category.

In [ ]:
df_incident[df_incident['incident_id'].duplicated(keep=False)][['incident_id', 'category', 'subcategory']].sort_values('incident_id').head(50)

In [ ]:
sorted(list(df_incident['category'].unique()))

### There is the same category, but with different British and American spellings: 'Weapons Offence' versus 'Weapons Offense.'

In [ ]:
df_incident.loc[df_incident['category']=='Weapons Offense', 'category'] = 'Weapons Offence'

### There are some categories I want to see their description to understand better. 

In [ ]:
df_incident[(df_incident['category'].isin(['Suspicious','Suspicious Occ'])) & \
    ~(df_incident['description'].isin(['Suspicious Occurrence','Suspicious Vehicle' ,'Suspicious Person','Suspicious Act Towards Female']) )]\
        [['subcategory', 'description']]

In [ ]:
df_incident[df_incident['category']=='Other Offenses']['description']

## Firsty I will drop the rows has crimes has no require immidiate police attention or no police attention at all

In [ ]:
df_incident = df_incident[(~df_incident['category'].isin(['Case Closure', 'Non-Criminal' , 'Offences Against The Family And Children','Courtesy Report', 'Fire Report','Recovered Vehicle'
                                          ,'Warrant', 'Liquor Laws', 'Other Miscellaneous', 'Gambling', 'Other', 'Missing Person', \
                                              'Civil Sidewalks', 'Traffic Collision', 'Suicide','Suspicious','Suspicious Occ', 'Other Offenses']))]

In [ ]:
sorted(list(df_incident['category'].unique()))

## Let's see crime distribution on map

In [ ]:
def find_on_map_with_category(cat_type):
    sf_map= folium.Map(location=[37.7749, -122.4194], zoom_start=12)

    marker_cluster = MarkerCluster().add_to(sf_map)
    
    for i, row in df_incident.loc[df_incident['category']==cat_type].iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['incident_date'].strftime('%B %d,%Y')
        ).add_to(marker_cluster)
    display(sf_map)


In [ ]:
display(df_incident[(df_incident['category']=='Malicious Mischief') & (df_incident['subcategory']=='Other')][['subcategory', 'description']].head())
df_incident[df_incident['category']=='Malicious Mischief'][['subcategory']].value_counts()

### Looks like we can Change `Malicious Mischief` to `Vandalism` after droping wher subcategory is Other

In [ ]:
df_incident.shape

In [ ]:
df_incident = df_incident[~((df_incident['category']=='Malicious Mischief') & (df_incident['subcategory']=='Other'))]
df_incident.loc[df_incident['category']=='Malicious Mischief', 'category'] = 'Vandalism'

In [ ]:
def find_on_map_with_subcategory(cat_type):
    sf_map= folium.Map(location=[37.7749, -122.4194], zoom_start=12)

    marker_cluster = MarkerCluster().add_to(sf_map)
    
    for i, row in df_incident.loc[df_incident['subcategory']==cat_type].iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['incident_date'].strftime('%B %d,%Y')
        ).add_to(marker_cluster)
    return(sf_map)
    
sf_map =find_on_map_with_subcategory('Larceny - From Vehicle')
sf_map.save('larceny_from_vehicle_on_map.html')

In [ ]:
display(sf_map)

### Wow! That is more than what I expected, I hope our prediction model will work!

In [ ]:
sorted_categories = df_incident['category'].value_counts().index.to_list()
sorted_categories

In [ ]:
df_incident.info()

In [ ]:
df_incident.head()

In [ ]:
# all the day name matches with the actual date
(~(df_incident['incident_day'] == df_incident['incident_date'].dt.day_name())).sum()

In [ ]:
df_incident['category'] = pd.Categorical(df_incident['category'], categories=sorted_categories, ordered=True)
df_incident['subcategory'] = pd.Categorical(df_incident['subcategory'])


In [ ]:
df_incident.shape

In [ ]:
df_incident = df_incident.sort_values(by=['incident_id', 'category'])
df_incident.drop_duplicates(subset='incident_id', keep='first', inplace=True)
df_incident.shape


In [ ]:
df_incident.sort_values(by='incident_date', inplace=True)
df_incident.drop(columns=['incident_id', 'incident_code'], inplace=True)
print(df_incident.shape)
df_incident.head()

In [ ]:
profile = ProfileReport(df_incident, title="Profiling Report")

In [ ]:
profile.to_file('sf_crime_clean_data_report.html')

In [ ]:
profile

In [ ]:
columns = df_incident.columns.to_list()
data_types = [df_incident[column].dtypes.name for column in columns]
data_type_dict = dict(zip(columns, data_types))

In [ ]:
df_incident.reset_index(drop=True, inplace=True)
df_incident.head()

In [ ]:
df_incident.to_csv(path+'/../data/interim/sf_crime_cleaned.csv', index=False)

In [ ]:
with open(path+'/../data/interim/sf_incident_dtypes.json', 'w') as f:
    json.dump(data_type_dict, f)